<a href="https://colab.research.google.com/github/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/blob/master/Tarea_de_Riesgo_de_Mercado_SH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%load_ext rpy2.ipython
!rm -rf Tarea-de-Riesgo-de-Mercado-SH
#https://github.com/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/blob/master/Tarea_de_Riesgo_de_Mercado_SH.ipynb
!git clone "https://github.com/Osvaldo1417/Tarea-de-Riesgo-de-Mercado-SH/"

Cloning into 'Tarea-de-Riesgo-de-Mercado-SH'...
remote: Enumerating objects: 794, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 794 (delta 86), reused 40 (delta 40), pack-reused 690
Receiving objects: 100% (794/794), 41.14 MiB | 31.42 MiB/s, done.
Resolving deltas: 100% (521/521), done.


Paso de carga de librerías en R de todas las que se utilizarán

In [9]:
%%R
install.packages("quantmod")
library(quantmod)

install.packages("data.table")
library(data.table)

install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)

library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

# Funciones

In [100]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

genera_esc <- function(lamda,w0,n){
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  return(p_esc)
}

# Carga de Datos

In [101]:
%%R
fval=as.Date("20210331",format="%Y%m%d") #Fecha de valoración
itpl=1 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo

## Acciones y Divisas

In [102]:
%%R
acc_names <- c("America","Carso","Walmart")
div_names <- c("Euros","Libras","Dolares")

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Sym_acciones<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )#tienen que ir en orden alfabético
pos_acc=c(-5000,1000,1200) #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
Sym_divisas<-c("EURUSD=X","GBPUSD=X","USDMXN=X" ) #tienen que ir en orden alfabético
pos_div=c(700,-600,1500) #monto inicial invertido en divisas
nh=3660 #días de historia

In [103]:
%%R
#CARGA DE DATOS DE ACCIONES
pos=cbind(t(pos_div),t(pos_acc))
start_date=fval-nh #fecha inicial
#Creación del objeto para guardar los datos
acciones<-new.env()
divisas<-new.env()
#obtener los datos
getSymbols.yahoo(Sym_acciones,env=acciones,from=start_date,to=fval)
getSymbols.yahoo(Sym_divisas,env=divisas,from=start_date,to=fval)

#limpiarlos, alinearnos y quedarnos con el precio de cierre 
bt.prep(acciones,align='remove.na',fill.gaps = T)
bt.prep(divisas,align='remove.na',fill.gaps=T)

precios_acc = acciones$prices
precios_div = divisas$prices

precios_div=cbind(precios_div[,1]*precios_div[,3],
                     precios_div[,2]*precios_div[,3],precios_div[,3])

precios_acc_div = merge(precios_acc,precios_div,join = "inner")
tail(precios_acc_div)

           AMXL.MX GCARSOA1.MX WALMEX.MX EURUSD.X GBPUSD.X USDMXN.X
2021-03-23   14.07       54.02     62.73 24.55267 28.52265 20.57539
2021-03-24   14.06       54.00     64.28 24.63700 28.57014 20.80791
2021-03-25   14.13       54.78     65.48 24.66923 28.58649 20.88300
2021-03-26   14.05       55.72     69.95 24.35762 28.43732 20.68303
2021-03-29   14.27       58.10     66.31 24.32163 28.45434 20.63253
2021-03-30   14.28       57.41     66.24 24.25697 28.38677 20.60738


In [104]:
%%R
aux2=data.table(Date=as.Date(index(precios_acc_div)),coredata(precios_acc_div))
colnames(aux2) <- c("Date","America","Carso","Walmart","Euros","Libras","Dolares")
tail(aux2)

         Date America Carso Walmart    Euros   Libras  Dolares
1: 2021-03-23   14.07 54.02   62.73 24.55267 28.52265 20.57539
2: 2021-03-24   14.06 54.00   64.28 24.63700 28.57014 20.80791
3: 2021-03-25   14.13 54.78   65.48 24.66923 28.58649 20.88300
4: 2021-03-26   14.05 55.72   69.95 24.35762 28.43732 20.68303
5: 2021-03-29   14.27 58.10   66.31 24.32163 28.45434 20.63253
6: 2021-03-30   14.28 57.41   66.24 24.25697 28.38677 20.60738


## Bonos

In [105]:
%%R
#CETES
tasa_guber_bcc="Tarea-de-Riesgo-de-Mercado-SH/tasa_guber.txt"
plazos_bcc=cbind(180)
contratos_bcc=cbind(1)
nominal_bcc=1500 #CETE

# BONO M
tasa_fija_bm="Tarea-de-Riesgo-de-Mercado-SH/tasa_yield.txt"
tfcupon_bm=cbind( 0.065) #Tasafija del cupón
plazos_bm=cbind( 3600) #Vencimiento del bono
plazocupon_bm=cbind(360) #plazos fijos de cada cupón
contratos_bm=cbind(1) #posición invertida
nominal_bm=1000

#BONDES D
base_tasa_desc="Tarea-de-Riesgo-de-Mercado-SH/tasa_guber_st.txt"
base_tasa_fondeo="Tarea-de-Riesgo-de-Mercado-SH/tfondeo.txt"
plazo_bdm= 707 #Vencimiento del bono
plazocupon_bdm = 28 #plazos_bdm fijos de cada cupón
contratos_bdm = -1 #posición invertida
nominal_bdm = 1000

In [106]:
%%R
#Bono Cupon CERO
data<-read.table(tasa_guber_bcc)
n<-nrow(data)
m_orig=ncol(data)
X_orig_bcc=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bcc=X_orig_bcc%>%select(-V1)
nodos_bcc=data.frame(data[1,2:m_orig])

## BONO M
data1<-read.table(tasa_fija_bm)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig_bm=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X_orig_bm=X_orig_bm%>%select(-V1)
nodos_bm=data.frame(data1[1,2:m_orig])
n=n-1

##BONDE
data1<-read.table(base)
n_bd<-nrow(data1)
m_bd=ncol(data1)
X1_orig=mutate(data.frame(data1[2:n,]), Date=as.Date(V1,format="%Y%m%d"))
X1_orig=X1_orig%>%select(-V1)
nodos1_bd=data.frame(data1[1,2:m_bd])
n_bd=n_bd-1

data2<-read.table(base_tasa_fondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,])
X2_orig_bd=mutate(X2_orig_bd, Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(V2))
X2_orig_bd=X2_orig_bd%>%select(-V1)
tfh=seq(min(X2_orig_bd$Date), max(X2_orig_bd$Date), "days")
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

data3<-read.table(base_tasa_desc)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := Date][order(-Date)]
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf]

tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

## Futuros

## Swap's

## Opciones

# Integración de insumos

In [107]:
%%R
fechas= data.table(Date=as.Date(aux2[X_orig_bcc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity y guber
fechas=data.table(Date=as.Date(fechas[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber y st (bonde)
fechas=data.table(Date=as.Date(fechas[X_orig_bm,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield (bono M) y st (bonde)
tail(fechas)

         Date
1: 2020-10-21
2: 2020-10-20
3: 2020-10-19
4: 2020-10-16
5: 2020-10-15
6: 2020-10-14


## Segreración de Datos

### Acciones y Divisas

In [109]:
%%R
n=nrow(fechas) #Historia de todos

#historia de acciones y divisas
precios_acc_div = fechas[aux2,on=.(Date),nomatch=0][order(-Date)]
precios_acc_div = precios_acc_div%>%select(-Date)
precios_acc_div

     America Carso Walmart    Euros   Libras  Dolares
  1:   14.28 57.41   66.24 24.25697 28.38677 20.60738
  2:   14.27 58.10   66.31 24.32163 28.45434 20.63253
  3:   14.05 55.72   69.95 24.35762 28.43732 20.68303
  4:   14.13 54.78   65.48 24.66923 28.58649 20.88300
  5:   14.06 54.00   64.28 24.63700 28.57014 20.80791
 ---                                                 
110:   13.88 42.42   51.90 24.95852 27.46057 21.20725
111:   13.48 44.08   50.91 24.70658 27.26307 21.08880
112:   13.48 44.08   50.91 24.91757 27.46087 21.28410
113:   13.64 45.25   50.71 25.04541 27.75038 21.32617
114:   13.79 46.25   50.77 25.05018 27.59543 21.33348


### Bonos

In [110]:
%%R
#historia de curva gubernamental
X_orig_bcc=fechas[X_orig_bcc,on=.(Date),nomatch=0][order(-Date)]
X_orig_bcc=X_orig_bcc%>%select(-Date)

## Historia Bono M
X_orig_bm=fechas[X_orig_bm,on=.(Date),nomatch=0][order(-Date)]
X_orig_bm=X_orig_bm%>%select(-Date)

#Historia de curvas de bonde
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X1_orig=fechas[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)

X2_orig_bd=fechas[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)

X3_orig_bd=fechas[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)

### Futuros

### Swap's

### Opciones

# Simulación Historica

## Acciones y Divisas

In [63]:
%%R
x0_acc_div=precios_acc_div[1,]
DeltaX_acc_div=as.matrix(log(precios_acc_div[1:(n-1)]/precios_acc_div[2:(n)]))
V0_acc_div=cbind(t(pos_acc),t(pos_div))*x0_acc_div
m_acc=length(pos_acc)
m_div=length(pos_div)

print("Precio Actual")
print(x0_acc_div)

print("Valor del portafolio")
print(V0_acc_div)

[1] "Precio Actual"
   America Carso Walmart    Euros   Libras  Dolares
1:   14.28 57.41   66.24 24.25697 28.38677 20.60738
[1] "Valor del portafolio"
   America Carso Walmart    Euros    Libras  Dolares
1:  -71400 57410   79488 16979.88 -17032.06 30911.07


## Bonos

In [140]:
%%R
## CETE
m_bcc=ncol(plazos_bcc)
X_bcc=matrix(0,n,m_bcc)
for (i in 1:n){
    X_bcc[i,]=if(itpl==0){approx(as.matrix(nodos_bcc),as.matrix(X_orig_bcc[i,]),plazos_bcc)$y}else{talamb(as.matrix(nodos_bcc),as.matrix(X_orig_bcc[i,]),plazos_bcc)}
}

bonocupcero = function(i,t)
{
1/(1+i*t/360)
}

x0_bcc=X_bcc[1,]
V0_bcc=bonocupcero(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc #Valor actual de cada bono
print("Valor del CETE")
print(V0_bcc)

##Bono M
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:(n)){
  X_bm[i,]=if(itpl==0){approx(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm)$y}else{talamb(as.matrix(nodos_bm),as.matrix(X_orig_bm[i,]),plazos_bm)}
}

#Función de valoración por tasa yield
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
N=as.integer(plazos/plazocupon)+1
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0_bm=X_bm[1,] #tasas de descuento valor actual
V0_bm = bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
print("Valor del Bono M")
print(V0_bm)


#BONDE D CÁLCULO
m=1

N_bd=as.integer(plazo_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo

plazini_bd=plazo_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos

for (j in (1:m)){
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}

for (j in (1:m)){
  if (j==1){
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else{
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazo_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n)){
  Xvp_bd[i,]=if(itpl==0){approx(as.matrix(nodos1_bd),as.matrix(X1_orig[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos1_bd),as.matrix(X1_orig[i,]),VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm,rule=2)$y}else{talamb(as.matrix(nodos3_bd),as.matrix(X3_orig_bd[i,]),VTplazos_bdm)}
}

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_orig_bd$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))

bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv){
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bd=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
print("Valor del BONDE")
print(V0_bd)


m_bcc <- ncol(X_bcc)
m_bm <- ncol(X_bm)
m_bd <- ncol(X_bd_ext)
X_bonos <- cbind(X_bcc,X_bm,X_bd_ext)
V0_bonos <- cbind(V0_bcc,V0_bm,V0_bd)

[1] "Valor del CETE"
         [,1]
[1,] 1462.448
[1] "Valor del Bono M"
         [,1]
[1,] 1108.391
[1] "Valor del BONDE"
          [,1]
[1,] -930.9849


## Futuros

## Swap's

## Opciones

# Medición de Riesgo


In [118]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 8 instrumentos financieros (9 si separamos acciones y divisas)
n_if=matrix(0,6,1)
n_if[1]=ncol(precios_acc_div) #acciones y divisas
n_if[2]=ncol(X_bonos) #bonde
#n_if[3]=ncol(X_sw) #swaps
#n_if[4]=ncol(X_oir) #opciones tasa de interés
#n_if[5]=ncol(X_futtdc) #Forwards de tipo de cambio
#n_if[6]=ncol(X_futind) #Forwards de índices
rownames(n_if) <- c("Acciones y Divisas","Bonos","Futuros TDC","Futuros IPC","Swap's","Opciones")

#valor del portafolios

#V0_port=cbind(V0_acc_div, V0_bd, V0_sw, V0_oir,V0_fwtdc,V0_fwind)
V0_port=cbind(V0_acc_div,V0_bonos)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
#X_port=cbind(precios_acc_div,X_bd_ext,X_sw,X_oir,X_futtdc,X_futind) #Factores de riesgo del portafolios de 8(9) instrumentos financieros
X_port=cbind(precios_acc_div,X_bonos) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
#alpha=0.98 #Nivel de Confianza para las medidas de riesgo

DeltaX_s=DeltaX_port
#print("Matriz de Rendimientos del Portafolio")
#print(head(DeltaX_s))
print("Número de Factores de Riesgo")
print(ncol(DeltaX_s))
print("Número de Datos")
print(nrow(DeltaX_s))
print("Número de Factores de Riesgo por Insumo")
print(n_if)

[1] "Número de Factores de Riesgo"
[1] 86
[1] "Número de Datos"
[1] 113
[1] "Número de Factores de Riesgo por Insumo"
                   [,1]
Acciones y Divisas    6
Bonos                80
Futuros TDC           0
Futuros IPC           0
Swap's                0
Opciones              0


## Acciones y Divisas

In [ ]:
%%R
m=m_acc+m_div    #PASO CLAVE
X_s_acc_div=matrix(0,Ns,m)
V_acc_div=matrix(0,Ns,m)
Vfr1_acc_div=matrix(0,Ns,m_acc)
Vfr2_acc_div=matrix(0,Ns,m_div)
PG_acc_div=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_acc_div=matrix(0,Ns,m_acc)
PGfr2_acc_div=matrix(0,Ns,m_div)
PGT_acc_div=matrix(0,Ns,1)
PGfr1T_acc_div=matrix(0,Ns,1)
PGfr2T_acc_div=matrix(0,Ns,1)

In [ ]:
%%R
DeltaX_s_acc_div=DeltaX_s[,(1:m)]
x0_acc_div=precios_acc_div[1,]

for (i in 1:Ns)
{
  X_s_acc_div[i,]=as.matrix(x0_acc_div*exp(DeltaX_s_acc_div[i,]))
  V_acc_div[i,]=cbind(t(pos_acc),t(pos_div))*X_s_acc_div[i,]

  Vfr1_acc_div[i,]=t(pos_acc)*X_s_acc_div[i,1:m_acc]
  Vfr2_acc_div[i,]=t(pos_div)*X_s_acc_div[i,(m_acc+1):m]

  PG_acc_div[i,]=as.matrix(V_acc_div[i,]-V0_acc_div)

  PGfr1_acc_div[i,]=as.matrix(Vfr1_acc_div[i,]-V0_acc_div[,1:m_acc])
  PGfr2_acc_div[i,]=as.matrix(Vfr2_acc_div[i,]-V0_acc_div[,(m_acc+1):m])

  PGT_acc_div[i,]=sum(PG_acc_div[i,])

  PGfr1T_acc_div[i,]=sum(PGfr1_acc_div[i,])
  PGfr2T_acc_div[i,]=sum(PGfr2_acc_div[i,])
}

In [ ]:
%%R
print("Perdida/Ganancias de acciones y divisas")
print(head(PG_acc_div))
print("Perdidas/Ganancias de acciones")
print(head(PGfr1_acc_div))
print("Perdidas/Ganancias de divisas")
print(head(PGfr2_acc_div))
print("Perdidas/Ganancias totales de acciones y divisas")
print(head(PGT_acc_div))
print("Perdidas/Ganancias totales de acciones")
print(head(PGfr1T_acc_div))
print("Perdidas/Ganancias totales de divisas")
print(head(PGfr2T_acc_div))

[1] "Perdida/Ganancias de acciones y divisas"
           [,1]        [,2]        [,3]       [,4]      [,5]       [,6]
[1,] -1882.9931   890.53688  1933.29002   30.01090  14.92095 -130.59748
[2,]  -255.0674  -761.22966  2201.76179   40.00016 109.62618 -186.47594
[3,]   101.9203 -1491.92700 -1539.27087   64.32902  38.91264   90.95452
[4,]  1408.0051    39.28302  1551.05679  -66.97251  51.32280  -53.71657
[5,]  -454.5194   157.45672    37.53721 -118.54352 136.32683 -236.36914
[6,]  -916.9320  1918.04583   718.46750  -67.82311 106.44992  -67.01014
[1] "Perdidas/Ganancias de acciones"
           [,1]        [,2]        [,3]
[1,] -1882.9931   890.53688  1933.29002
[2,]  -255.0674  -761.22966  2201.76179
[3,]   101.9203 -1491.92700 -1539.27087
[4,]  1408.0051    39.28302  1551.05679
[5,]  -454.5194   157.45672    37.53721
[6,]  -916.9320  1918.04583   718.46750
[1] "Perdidas/Ganancias de divisas"
           [,1]      [,2]       [,3]
[1,]   30.01090  14.92095 -130.59748
[2,]   40.00016 109.626

In [ ]:
%%R
w0=1-alpha
n = Ns
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,n)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01960000
[3,] 0.01920800
[4,] 0.01882384
[5,] 0.01844736
[6,] 0.01807842
                [,1]
[2509,] 1.977096e-24
[2510,] 1.937554e-24
[2511,] 1.898803e-24
[2512,] 1.860827e-24
[2513,] 1.823611e-24
[2514,] 1.787139e-24
[1] "Validación de los pesos"
[1] 1


In [ ]:
%%R
#VaR por posición
VaRCont_acc_div=matrix(0,1,m)
VaRfr1_acc_div=matrix(0,1,m_acc)
VaRfr2_acc_div=matrix(0,1,m_div)

CVaRCont_acc_div=matrix(0,1,m)
CVaRfr1_acc_div=matrix(0,1,m_acc)
CVaRfr2_acc_div=matrix(0,1,m_div)

VaRCont_CA_acc_div=matrix(0,1,m)
VaRfr1_CA_acc_div=matrix(0,1,m_acc)
VaRfr2_CA_acc_div=matrix(0,1,m_div)

CVaRCont_CA_acc_div=matrix(0,1,m)
CVaRfr1_CA_acc_div=matrix(0,1,m_acc)
CVaRfr2_CA_acc_div=matrix(0,1,m_div)

In [ ]:
%%R
for (i in (1:m))
{
  #VaR de accines y divisas sin alisado
  VaRCont_acc_div[i]=equantile(PG_acc_div[,i],1-alpha,Ns)
  #CVaR de acciones y divisas sin alisado
  CVaRCont_acc_div[i]= mean(merge(which(PG_acc_div[,i]<VaRCont_acc_div[i]),cbind(seq(1,Ns),PG_acc_div[,i]), by.x=1,by.y=1)[,2])
  #VaR de acciones y divisas con alisado
  VaRCont_CA_acc_div[i]=wquantile(PG_acc_div[,i],p_esc, 1-alpha)
  #CVaR de acciones y divisas con alisado
  CVaRCont_CA_acc_div[i]= wcvar(PG_acc_div[,i],p_esc, 1-alpha)
}

#VaR por acciones
VaRfr1_acc_div <- as.matrix(t(VaRCont_acc_div[1:m_acc]))
#VaR por divisas
VaRfr2_acc_div <- as.matrix(t(VaRCont_acc_div[(m_acc+1):m]))

#CVar por acciones
CVaRfr1_acc_div <- as.matrix(t(CVaRCont_acc_div[1:m_acc]))
#CVar por divisas
CVaRfr2_acc_div <- as.matrix(t(CVaRCont_acc_div[(m_acc+1):m]))

#Var con alisado por acciones
VaRfr1_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[1:m_acc]))
#Var con alsiado por divisas
VaRfr2_CA_acc_div <- as.matrix(t(VaRCont_CA_acc_div[(m_acc+1):m]))

#CVar con alisado por acciones
CVaRfr1_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[1:m_acc]))
#CVar con alisado por divisas
CVaRfr2_CA_acc_div <- as.matrix(t(CVaRCont_CA_acc_div[(m_acc+1):m]))

#Var sin alisado total
VaRTotal_acc_div=equantile(PGT_acc_div,1-alpha,Ns)
VaRTotalfr1_acc_div=equantile(PGfr1T_acc_div,1-alpha,Ns)
VaRTotalfr2_acc_div=equantile(PGfr2T_acc_div,1-alpha,Ns)

#CVar sin alisado total
CVaRTotal_acc_div= mean(merge(which(PGT_acc_div<VaRTotal_acc_div),cbind(seq(1,Ns),PGT_acc_div), by.x=1,by.y=1)[,2])
CVaRTotalfr1_acc_div= mean(PGfr1T_acc_div[which(PGfr1T_acc_div<VaRTotalfr1_acc_div),])
CVaRTotalfr2_acc_div= mean(PGfr2T_acc_div[which(PGfr2T_acc_div<VaRTotalfr2_acc_div),])

#Var con alisado total
VaRTotal_CA_acc_div=wquantile(PGT_acc_div,p_esc,1-alpha)
VaRTotalfr1_CA_acc_div=wquantile(PGfr1T_acc_div,p_esc,1-alpha)
VaRTotalfr2_CA_acc_div=wquantile(PGfr2T_acc_div,p_esc,1-alpha)

#CVar con alisado total
CVaRTotal_CA_acc_div= wcvar(PGT_acc_div,p_esc,1-alpha)
CVaRTotalfr1_CA_acc_div= wcvar(PGfr1T_acc_div,p_esc,1-alpha)
CVaRTotalfr2_CA_acc_div= wcvar(PGfr2T_acc_div,p_esc,1-alpha)

### Valor en Riesgo

In [ ]:
%%R
colnames(VaRCont_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_acc_div) <- acc_names
colnames(VaRfr2_acc_div) <- div_names
colnames(VaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(VaRfr1_CA_acc_div) <- acc_names
colnames(VaRfr2_CA_acc_div) <- div_names

colnames(CVaRCont_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_acc_div) <- acc_names
colnames(CVaRfr2_acc_div) <- div_names
colnames(CVaRCont_CA_acc_div) <- c(acc_names,div_names)
colnames(CVaRfr1_CA_acc_div) <- acc_names
colnames(CVaRfr2_CA_acc_div) <- div_names

In [ ]:
%%R
print("VaR por acciones y divisas")
VaR_acc_div <- as.matrix(rbind(VaRCont_acc_div,VaRCont_CA_acc_div,V0_acc_div))
rownames(VaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(VaR_acc_div)

print("VaR Total")
VaR_Total_acc_div <- as.matrix(rbind(cbind(VaRTotal_acc_div,VaRTotalfr1_acc_div,VaRTotalfr2_acc_div),
                                     cbind(VaRTotal_CA_acc_div,VaRTotalfr1_CA_acc_div,VaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(VaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(VaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(VaR_Total_acc_div)

[1] "VaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -3349.254 -3017.479 -3146.197  -292.0318   -282.2936
Con alisado        -3487.312 -2990.556 -3177.200  -156.9451   -134.5073
Valor portafolio -105600.005 74410.004 98303.998 15531.3719 -15670.5984
                    Dolares
Sin alisado       -453.4187
Con alisado       -356.7110
Valor portafolio 29817.2265
[1] "VaR Total"
                     Total  Acciones    Divisas
Sin alisado      -4571.742 -4704.415  -503.0257
Con alisado      -4949.976 -4973.593  -379.4284
Valor portafolio 96791.997 67113.997 29677.9999


### Valor en Riesgo Condicional

In [ ]:
%%R
print("CVaR por acciones y divisas")
CVaR_acc_div <- as.matrix(rbind(CVaRCont_acc_div,CVaRCont_CA_acc_div,V0_acc_div))
rownames(CVaR_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
print(CVaR_acc_div)

print("CVaR Total")
CVaR_Total_acc_div <- as.matrix(rbind(cbind(CVaRTotal_acc_div,CVaRTotalfr1_acc_div,CVaRTotalfr2_acc_div),
                                     cbind(CVaRTotal_CA_acc_div,CVaRTotalfr1_CA_acc_div,CVaRTotalfr2_CA_acc_div),
                                     cbind(sum(V0_acc_div),sum(V0_acc_div[,1:m_acc]),sum(V0_acc_div[,(m_acc+1):m]))))
rownames(CVaR_Total_acc_div) <- c("Sin alisado","Con alisado","Valor portafolio")
colnames(CVaR_Total_acc_div) <- c("Total","Acciones","Divisas")
print(CVaR_Total_acc_div)

[1] "CVaR por acciones y divisas"
                     America     Carso   Walmart      Euros      Libras
Sin alisado        -4528.041 -4166.973 -4428.774  -356.0485   -419.0875
Con alisado        -4024.923 -3692.181 -3434.240  -215.0101   -164.5758
Valor portafolio -105600.005 74410.004 98303.998 15531.3719 -15670.5984
                    Dolares
Sin alisado       -598.4633
Con alisado       -578.8262
Valor portafolio 29817.2265
[1] "CVaR Total"
                     Total  Acciones    Divisas
Sin alisado      -6341.516 -6511.754  -638.6302
Con alisado      -6050.123 -6090.790  -473.4383
Valor portafolio 96791.997 67113.997 29677.9999


## Bonos

In [172]:
%%R
#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=count(N_bd)    #PASO CLAVE
X_s_bon=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
Vfr1_bon=matrix(0,Ns,m) #BCC
Vfr2_bon=matrix(0,Ns,m) #BM
Vfr3_bon=matrix(0,Ns,m) #BD fonde
Vfr4_bon=matrix(0,Ns,m) #BD gober
Vfr5_bon=matrix(0,Ns,m) #BD desc

PGfr1_bon=matrix(0,Ns,m)
PGfr2_bon=matrix(0,Ns,m)
PGfr3_bon=matrix(0,Ns,m)
PGfr4_bon=matrix(0,Ns,m)
PGfr5_bon=matrix(0,Ns,m)

PGT_bon=matrix(0,Ns,1)
PGfr1T_bon=matrix(0,Ns,1)
PGfr2T_bon=matrix(0,Ns,1)
PGfr3T_bon=matrix(0,Ns,1)

In [177]:
%%R
DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bon=X_bonos[1,]    #PASO CLAVE

for (i in 1:Ns){
  X_s_bon[i,]=x0_bon*exp(DeltaX_s_bd[i,])
  
  Vfr1_bon[i,] = bonocupcero(X_s_bon[i,m_bcc],plazos_bcc)*contratos_bcc*nominal_bcc
  Vfr2_bon[i,] = bonoMyield(X_s_bon[i,(m_bcc+1)],plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)
  Vfr3_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, X_s_bon[i,(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm,
                        x0_bon[((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], x0_bon[((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)
  Vfr4_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, x0_bon[(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm, 
                        X_s_bon[i,((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], x0_bon[((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)
  Vfr5_bon[i,] = bondeD(contratos_bdmT, nominal_bdm, x0_bon[(m_bcc+m_bm+1)], plazocupon_bdmT, VTplazos_bdm,
                        x0_bon[((n_if[2]-m_bcc-m_bm)/3+1+m_bcc+m_bm):((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm)], X_s_bon[i,((n_if[2]-m_bcc-m_bm)*2/3+m_bcc+m_bm+1):(n_if[2])], N_bd,ddv)

  PGfr1_bon[i,]=Vfr1_bon[i,]-V0_bonos[m_bcc]
  PGfr2_bon[i,]=Vfr2_bon[i,]-V0_bonos[m_bcc+m_bm]
  PGfr3_bon[i,]=Vfr3_bon[i,]-V0_bonos[m_bcc+m_bm+1]
  PGfr4_bon[i,]=Vfr4_bon[i,]-V0_bonos[m_bcc+m_bm+1]
  PGfr5_bon[i,]=Vfr5_bon[i,]-V0_bonos[m_bcc+m_bm+1]

  PGfr1T_bon[i,] = sum(PGfr1_bon[i,])
  PGfr2T_bon[i,] = sum(PGfr2_bon[i,])
  PGfr3T_bon[i,] = sum(PGfr3_bon[i,])+sum(PGfr4_bon[i,])+sum(PGfr5_bon[i,])
  PGT_bon[i,] = PGfr1T_bon[i,]+PGfr2T_bon[i,]+PGfr3T_bon[i,]
}

In [180]:
%%R
PG_bonos <- cbind(PGfr1_bon,PGfr2_bon,PGfr3_bon,PGfr4_bon,PGfr5_bon)
colnames(PG_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
print("Perdida/Ganancia por Factor de Riesgo")
print(head(PG_bonos))

PGT_bonos <- cbind(PGT_bon,PGfr1T_bon,PGfr2T_bon,PGfr3T_bon)
colnames(PGT_bonos) <- c("Total","CETE","Bono M","BONDE")
print("Perdida/Ganancia Total")
print(head(PGT_bonos))

[1] "Perdida/Ganancia por Factor de Riesgo"
           CETE    Bono M BONDE Fondeo BONDE Gober BONDE Desc
[1,] 0.00000000  3.516363     4.563077    4.311145   4.471424
[2,] 0.00000000 -1.056845     4.745124    4.833375   4.661328
[3,] 0.07959504  1.041584     4.925349    4.383315   4.497853
[4,] 0.00000000 -4.794241     4.198979    5.260843   4.816544
[5,] 0.07942309  4.366633     6.683617    4.272529   4.457976
[6,] 0.07925116 -1.516938     2.936852    5.229067   4.804977
[1] "Perdida/Ganancia Total"
         Total       CETE    Bono M    BONDE
[1,] 16.862009 0.00000000  3.516363 13.34565
[2,] 13.182982 0.00000000 -1.056845 14.23983
[3,] 14.927695 0.07959504  1.041584 13.80652
[4,]  9.482125 0.00000000 -4.794241 14.27637
[5,] 19.860178 0.07942309  4.366633 15.41412
[6,] 11.533210 0.07925116 -1.516938 12.97090


In [187]:
%%R
w0=1-alpha
lambda =uniroot(function(x) w0*(1-x^(Ns))/(1-x)-1, c(0,0.99), tol = 1e-28)$root

p_esc=genera_esc(lambda,w0,Ns)
print("Pesos para el Alisado")
print(head(p_esc))
print(tail(p_esc))
print("Validación de los pesos")
sum(p_esc) #validamos que sume 1

[1] "Pesos para el Alisado"
           [,1]
[1,] 0.02000000
[2,] 0.01965647
[3,] 0.01931883
[4,] 0.01898700
[5,] 0.01866086
[6,] 0.01834033
              [,1]
[108,] 0.003132576
[109,] 0.003078769
[110,] 0.003025886
[111,] 0.002973911
[112,] 0.002922829
[113,] 0.002872624
[1] "Validación de los pesos"
[1] 1


In [188]:
%%R
#VaR por posición
VaRfr1_bd=matrix(0,1,m)
VaRfr2_bd=matrix(0,1,m)
VaRfr3_bd=matrix(0,1,m)
VaRfr4_bd=matrix(0,1,m)
VaRfr5_bd=matrix(0,1,m)

CVaRfr1_bd=matrix(0,1,m)
CVaRfr2_bd=matrix(0,1,m)
CVaRfr3_bd=matrix(0,1,m)
CVaRfr4_bd=matrix(0,1,m)
CVaRfr5_bd=matrix(0,1,m)

VaRfr1_CA_bd=matrix(0,1,m)
VaRfr2_CA_bd=matrix(0,1,m)
VaRfr3_CA_bd=matrix(0,1,m)
VaRfr4_CA_bd=matrix(0,1,m)
VaRfr5_CA_bd=matrix(0,1,m)

CVaRfr1_CA_bd=matrix(0,1,m)
CVaRfr2_CA_bd=matrix(0,1,m)
CVaRfr3_CA_bd=matrix(0,1,m)
CVaRfr4_CA_bd=matrix(0,1,m)
CVaRfr5_CA_bd=matrix(0,1,m)

In [212]:
%%R
for (i in (1:m))
{
  VaRfr1_bd[i]=equantile(PGfr1_bon[,i],1-alpha,Ns)
  VaRfr2_bd[i]=equantile(PGfr2_bon[,i],1-alpha,Ns)
  VaRfr3_bd[i]=equantile(PGfr3_bon[,i],1-alpha,Ns)
  VaRfr4_bd[i]=equantile(PGfr4_bon[,i],1-alpha,Ns)
  VaRfr5_bd[i]=equantile(PGfr5_bon[,i],1-alpha,Ns)
  
  CVaRfr1_bd[i]= mean(merge(which(PGfr1_bon[,i]<VaRfr1_bd[i]),cbind(seq(1,Ns),PGfr1_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_bd[i]= mean(merge(which(PGfr2_bon[,i]<VaRfr2_bd[i]),cbind(seq(1,Ns),PGfr2_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_bd[i]= mean(merge(which(PGfr3_bon[,i]<VaRfr3_bd[i]),cbind(seq(1,Ns),PGfr3_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr4_bd[i]= mean(merge(which(PGfr4_bon[,i]<VaRfr4_bd[i]),cbind(seq(1,Ns),PGfr4_bon[,i]), by.x=1,by.y=1)[,2])
  CVaRfr5_bd[i]= mean(merge(which(PGfr5_bon[,i]<VaRfr5_bd[i]),cbind(seq(1,Ns),PGfr5_bon[,i]), by.x=1,by.y=1)[,2])
  
  VaRfr1_CA_bd[i]=wquantile(PGfr1_bon[,i],p_esc, 1-alpha)
  VaRfr2_CA_bd[i]=wquantile(PGfr2_bon[,i],p_esc, 1-alpha)
  VaRfr3_CA_bd[i]=wquantile(PGfr3_bon[,i],p_esc, 1-alpha)
  VaRfr4_CA_bd[i]=wquantile(PGfr4_bon[,i],p_esc, 1-alpha)
  VaRfr5_CA_bd[i]=wquantile(PGfr5_bon[,i],p_esc, 1-alpha)

  CVaRfr1_CA_bd[i]= wcvar(PGfr1_bon[,i],p_esc, 1-alpha)
  CVaRfr2_CA_bd[i]= wcvar(PGfr2_bon[,i],p_esc, 1-alpha)
  CVaRfr3_CA_bd[i]= wcvar(PGfr3_bon[,i],p_esc, 1-alpha)
  CVaRfr4_CA_bd[i]= wcvar(PGfr4_bon[,i],p_esc, 1-alpha)
  CVaRfr5_CA_bd[i]= wcvar(PGfr5_bon[,i],p_esc, 1-alpha)
}


VaRT_bd=equantile(PGT_bon[,i],1-alpha,Ns)
VaRfr1T_bd=equantile(PGfr1T_bon[,i],1-alpha,Ns)
VaRfr2T_bd=equantile(PGfr2T_bon[,i],1-alpha,Ns)
VaRfr3T_bd=equantile(PGfr3T_bon[,i],1-alpha,Ns)

CVaRT_bd= mean(merge(which(PGT_bon[,i]<VaRT_bd[i]),cbind(seq(1,Ns),PGT_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr1T_bd= mean(merge(which(PGfr1T_bon[,i]<VaRfr1T_bd[i]),cbind(seq(1,Ns),PGfr1T_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr2T_bd= mean(merge(which(PGfr2T_bon[,i]<VaRfr2T_bd[i]),cbind(seq(1,Ns),PGfr2T_bon[,i]), by.x=1,by.y=1)[,2])
CVaRfr3T_bd= mean(merge(which(PGfr3T_bon[,i]<VaRfr3T_bd[i]),cbind(seq(1,Ns),PGfr3T_bon[,i]), by.x=1,by.y=1)[,2])

VaRT_CA_bd=wquantile(PGT_bon[,i],p_esc, 1-alpha)
VaRfr1T_CA_bd=wquantile(PGfr1T_bon[,i],p_esc, 1-alpha)
VaRfr2T_CA_bd=wquantile(PGfr2T_bon[,i],p_esc, 1-alpha)
VaRfr3T_CA_bd=wquantile(PGfr3T_bon[,i],p_esc, 1-alpha)

CVaRT_CA_bd= wcvar(PGT_bon[,i],p_esc, 1-alpha)
CVaRfr1T_CA_bd= wcvar(PGfr1T_bon[,i],p_esc, 1-alpha)
CVaRfr2T_CA_bd= wcvar(PGfr2T_bon[,i],p_esc, 1-alpha)
CVaRfr3T_CA_bd= wcvar(PGfr3T_bon[,i],p_esc, 1-alpha)

### Valor en Riesgo

In [213]:
%%R
VaR_bonos <- rbind(cbind(VaRfr1_bd,VaRfr2_bd,VaRfr3_bd,VaRfr4_bd,VaRfr5_bd),
                   cbind(VaRfr1_CA_bd,VaRfr2_CA_bd,VaRfr3_CA_bd,VaRfr4_CA_bd,VaRfr5_CA_bd))
colnames(VaR_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
rownames(VaR_bonos) <- c("Sin alisado","Con alisado")
print("VaR por Bonos")
print(VaR_bonos)

VaRT_bonos <- rbind(cbind(VaRT_bd,VaRfr1T_bd,VaRfr2T_bd,VaRfr3T_bd),
                   cbind(VaRT_CA_bd,VaRfr1T_CA_bd,VaRfr2T_CA_bd,VaRfr3T_CA_bd),
                    cbind(sum(V0_bonos),V0_bonos))
colnames(VaRT_bonos) <- c("Total","CETE","Bono M","BONDE")
rownames(VaRT_bonos) <- c("Sin alisado","Con alisado","Valor portafolio")
print("VaR Total")
print(VaRT_bonos)

[1] "VaR por Bonos"
                  CETE    Bono M BONDE Fondeo BONDE Gober BONDE Desc
Sin alisado -0.7007460 -17.40393     3.077623    2.810776   3.929287
Con alisado -0.9713551 -15.93927     2.936852    2.957170   3.981638
[1] "VaR Total"
                        Total         CETE     Bono M      BONDE
Sin alisado        -1.7024172   -0.7007460  -17.40393   10.60376
Con alisado        -0.2655804   -0.9713551  -15.93927   11.35086
Valor portafolio 1639.8544480 1462.4478733 1108.39143 -930.98486


### Valor en Riesgo Condicional

In [214]:
%%R
CVaR_bonos <- rbind(cbind(CVaRfr1_bd,CVaRfr2_bd,CVaRfr3_bd,CVaRfr4_bd,CVaRfr5_bd),
                   cbind(CVaRfr1_CA_bd,CVaRfr2_CA_bd,CVaRfr3_CA_bd,CVaRfr4_CA_bd,CVaRfr5_CA_bd))
colnames(CVaR_bonos) <- c("CETE","Bono M","BONDE Fondeo","BONDE Gober","BONDE Desc")
rownames(CVaR_bonos) <- c("Sin alisado","Con alisado")
print("CVaR por Bonos")
print(CVaR_bonos)

CVaRT_bonos <- rbind(cbind(CVaRT_bd,CVaRfr1T_bd,CVaRfr2T_bd,CVaRfr3T_bd),
                   cbind(CVaRT_CA_bd,CVaRfr1T_CA_bd,CVaRfr2T_CA_bd,CVaRfr3T_CA_bd),
                    cbind(sum(V0_bonos),V0_bonos))
colnames(CVaRT_bonos) <- c("Total","CETE","Bono M","BONDE")
rownames(CVaRT_bonos) <- c("Sin alisado","Con alisado","Valor portafolio")
print("CVaR Total")
print(CVaRT_bonos)

[1] "CVaR por Bonos"
                 CETE    Bono M BONDE Fondeo BONDE Gober BONDE Desc
Sin alisado -1.161487 -17.69106     2.738924    2.438372   3.793651
Con alisado -1.351619 -17.03967     2.540996    2.606643   3.855133
[1] "CVaR Total"
                       Total        CETE     Bono M       BONDE
Sin alisado        -1.803003   -1.161487  -17.69106    9.604497
Con alisado        -1.486992   -1.351619  -17.03967   10.269329
Valor portafolio 1639.854448 1462.447873 1108.39143 -930.984860
